In [96]:
import folium
import pandas
from geopy.geocoders import Nominatim
import branca.colormap as cm

from plugins import FullScreen
locations_dir = './locations/'

In [100]:
# Get Lat and Long of cities
lats_longs = []
cities = []
df = pandas.read_csv('{}cities.csv'.format(locations_dir), sep=',')
rows = df.shape[0]

for x in range(rows):
    cities.append(df.iloc[x]['City'])

geolocator = Nominatim()
for city in cities:
    location = geolocator.geocode(city)
    lats_longs.append([location.latitude, location.longitude])
    

In [102]:
# Sanity check
for result in lats_longs:
    print(result[0], result[1])
    

29.7589382 -95.3676974
29.4246002 -98.4951405
33.5778631 -101.8551665
32.7762719 -96.7968559
29.299328 -94.7945882
36.1662859 -115.149225
34.054935 -118.244476
22.2793278 114.1628131
39.9059631 116.391248
31.2253441 121.4888922
34.6968642 139.4049033
9.0 38.75
11.122604 39.634982
52.3745403 4.89797550561798


In [106]:
travel_map = folium.Map(location=[0, 0], zoom_start=2.4, prefer_canvas=True) # Instantiate a world map
travel_map.add_child(FullScreen.Fullscreen()) # Add a Fullscreen button
#travel_map.add_child(folium.LayerControl()) # Add Layer controls
travel_map.add_child(folium.features.LatLngPopup()) # Add Latitude Longitude Popup
#travel_map.add_child(folium.features.PolyLine([31,-99]))

# Colorscale: Steps of 10 between 0 and 200
colorscale = cm.linear.Set1.scale(0, 100).to_step(5)
colorscale.caption = 'Total number of days spent in each location'
travel_map.add_child(colorscale)

# Define locations: (Location, Years visited, total days visited, cities)
my_locations = [('CA', [2013], 7, "Los Angeles"),
                ('CHN', [2017], 14, "Beijing & Shanghai"),
                ('ETH', [2008, 2011], 105, "Addis Ababa & Dessie"),
                ('HK', [2017], 112, "Hong Kong"),
                ('JPN', [2017], 7, "Tokyo"),
                ('NLD', [2008, 2011], 2, "Holland"),
                ('NV', [2013], 7, "Las Vegas"),
                ('TX', [1996], -1, "I live here and I hate it")]

# Add GeoJson shapes to the map
for location in my_locations:
    map_loc = folium.GeoJson(open('{}{}.geo.json'.format(locations_dir, location[0]), encoding = "utf-8-sig").read(), 
                             style_function=lambda x: {'fillColor': 'yellow', 'dashArray': '5, 5'}, 
                             highlight_function=lambda x: {'weight': 2, 'fillColor': 'blue'})
    map_loc.add_child(folium.Popup(location[3])) # Show's the cities I've been to on click 
    travel_map.add_child(map_loc)
    
# Add Markers
for x in range(rows):
    folium.Marker([lats_longs[x][0], lats_longs[x][1]], popup=df.iloc[x]['City'], icon=folium.Icon(color='red',icon='info-sign')).add_to(travel_map)

In [107]:
travel_map